In [1]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from lime.lime_tabular import LimeTabularExplainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report



In [2]:
hotels=pd.read_csv("C:\\Users\\abdel\\ACL\\archive\\hotels.csv")
reviews=pd.read_csv("C:\\Users\\abdel\\ACL\\archive\\reviews.csv")
users=pd.read_csv("C:\\Users\\abdel\\ACL\\archive\\users.csv")

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE

# Load and merge data (assuming you already have this)
# hotels = pd.read_csv("hotels.csv")
# reviews = pd.read_csv("reviews.csv")
# users = pd.read_csv("users.csv")
# ... merging code ...
review_hotel_df=reviews.merge(hotels,on='hotel_id',how='left')
df=review_hotel_df.merge(users,on='user_id',how='left')

# Assuming df is your merged dataframe
# Define country groups
groups = {
    'North_America': ['United States', 'Canada'],
    'Western_Europe': ['Germany', 'France', 'United Kingdom', 'Netherlands', 'Spain', 'Italy'],
    'Eastern_Europe': ['Russia'],
    'East_Asia': ['China', 'Japan', 'South Korea'],
    'Southeast_Asia': ['Thailand', 'Singapore'],
    'Middle_East': ['United Arab Emirates', 'Turkey'],
    'Africa': ['Egypt', 'Nigeria', 'South Africa'],
    'Oceania': ['Australia', 'New Zealand'],
    'South_America': ['Brazil', 'Argentina'],
    'South_Asia': ['India'],
    'North_America_Mexico': ['Mexico']
}
df.rename(columns={"country_x":"hotel_country","country_y":"user_country"},inplace=True)
# Create country_group from hotel_country
df["country_group"] = df["hotel_country"].apply(
    lambda x: next((key for key, value in groups.items() if x in value), "Other")
)

# ===== FEATURE ENGINEERING =====

# 1. Score deviations from base metrics (if you have base metrics)
if 'cleanliness_base' in df.columns:
    df['cleanliness_deviation'] = df['score_cleanliness'] - df['cleanliness_base']
    df['comfort_deviation'] = df['score_comfort'] - df['comfort_base']
    df['facilities_deviation'] = df['score_facilities'] - df['facilities_base']

# 2. Aggregate score features
df['score_variance'] = df[['score_cleanliness', 'score_comfort', 'score_facilities', 
                            'score_location', 'score_staff', 'score_value_for_money']].var(axis=1)
df['score_mean'] = df[['score_cleanliness', 'score_comfort', 'score_facilities', 
                        'score_location', 'score_staff', 'score_value_for_money']].mean(axis=1)

# 3. Quality-to-star rating ratio
df['quality_per_star'] = df['score_overall'] / (df['star_rating'] + 0.1)

# 4. Value indicator (high score with good value for money)
df['value_indicator'] = df['score_overall'] * df['score_value_for_money']

# ===== ENCODE CATEGORICAL VARIABLES =====

# Encode traveller_type (DON'T use user_country to avoid leakage!)
df = pd.get_dummies(df, columns=['traveller_type'], drop_first=True)

# Encode gender
df = pd.get_dummies(df, columns=['user_gender'], drop_first=True)

# Encode age_group ordinally
age_order = {'18-24': 1, '25-34': 2, '35-44': 3, '45-54': 4, '55+': 5}
df['age'] = df['age_group'].map(age_order)

# ===== SELECT FEATURES (NO USER_COUNTRY!) =====

feature_columns = [
    # Review scores
    'score_overall', 'score_cleanliness', 'score_comfort', 'score_facilities',
    'score_location', 'score_staff', 'score_value_for_money',
    
    # Hotel features
    'star_rating',
    
    # Engineered features
    'score_variance', 'score_mean', 'quality_per_star', 'value_indicator',
    
    # User demographics (NO country!)
    'age',
    'traveller_type_Couple', 'traveller_type_Family', 'traveller_type_Solo',
    'user_gender_Male', 'user_gender_Other'
]

# Add deviation features if they exist
if 'cleanliness_deviation' in df.columns:
    feature_columns.extend(['cleanliness_deviation', 'comfort_deviation', 'facilities_deviation'])

# Remove any features that don't exist
feature_columns = [col for col in feature_columns if col in df.columns]

X = df[feature_columns]
y = df['country_group']

# ===== TRAIN-TEST SPLIT =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ===== FEATURE SCALING =====
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===== HANDLE CLASS IMBALANCE =====
# Use SMOTE to balance classes
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print(f"Original training set size: {len(X_train)}")
print(f"Balanced training set size: {len(X_train_balanced)}")
print(f"\nClass distribution after SMOTE:")
print(pd.Series(y_train_balanced).value_counts().sort_index())

# ===== TRAIN MODEL =====
log_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',  # Additional safety net for imbalance
    C=0.1,  # Add regularization
    solver='saga',  # Better for large datasets
    random_state=42
)

log_model.fit(X_train_balanced, y_train_balanced)

# ===== EVALUATE =====
y_pred = log_model.predict(X_test_scaled)

print("\n=== Improved Logistic Regression Evaluation ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print("\nDetailed Report:")
print(classification_report(y_test, y_pred, zero_division=0))

# ===== FEATURE IMPORTANCE =====
# Get feature importance (coefficients)
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': np.abs(log_model.coef_).mean(axis=0)
}).sort_values('importance', ascending=False)

print("\n=== Top 10 Most Important Features ===")
print(feature_importance.head(10))

Original training set size: 40000
Balanced training set size: 104511

Class distribution after SMOTE:
country_group
Africa                  9501
East_Asia               9501
Eastern_Europe          9501
Middle_East             9501
North_America           9501
North_America_Mexico    9501
Oceania                 9501
South_America           9501
South_Asia              9501
Southeast_Asia          9501
Western_Europe          9501
Name: count, dtype: int64

=== Improved Logistic Regression Evaluation ===
Accuracy: 0.5768
Precision: 0.6444
Recall: 0.5768
F1 Score: 0.5828

Detailed Report:
                      precision    recall  f1-score   support

              Africa       0.82      0.52      0.64      1226
           East_Asia       0.44      0.32      0.37      1216
      Eastern_Europe       0.34      0.92      0.49       394
         Middle_East       0.96      0.48      0.64       797
       North_America       0.59      0.67      0.62       792
North_America_Mexico       0.85 

In [4]:
review_hotel_df=reviews.merge(hotels,on='hotel_id',how='left')
df=review_hotel_df.merge(users,on='user_id',how='left')
df.rename(columns={"country_x":"hotel_country","country_y":"user_country"},inplace=True)


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Assuming you have already merged and created df as in your original code
# df = review_hotel_df.merge(users, on='user_id', how='left')
# df.rename(columns={"country_x":"hotel_country","country_y":"user_country"}, inplace=True)

# Define country groups
groups = {
    'North_America': ['United States', 'Canada'],
    'Western_Europe': ['Germany', 'France', 'United Kingdom', 'Netherlands', 'Spain', 'Italy'],
    'Eastern_Europe': ['Russia'],
    'East_Asia': ['China', 'Japan', 'South Korea'],
    'Southeast_Asia': ['Thailand', 'Singapore'],
    'Middle_East': ['United Arab Emirates', 'Turkey'],
    'Africa': ['Egypt', 'Nigeria', 'South Africa'],
    'Oceania': ['Australia', 'New Zealand'],
    'South_America': ['Brazil', 'Argentina'],
    'South_Asia': ['India'],
    'North_America_Mexico': ['Mexico']
}

df["country_group"] = df["hotel_country"].apply(
    lambda x: next((key for key, value in groups.items() if x in value), "Other")
)

print(f"Total reviews: {len(df)}")
print(f"\nCountry group distribution:")
print(df['country_group'].value_counts().sort_index())

# ===== FEATURE ENGINEERING =====

# 1. Score-based features (quality variance)
df['score_std'] = df[['score_cleanliness', 'score_comfort', 'score_facilities',
                       'score_location', 'score_staff', 'score_value_for_money']].std(axis=1)
df['score_mean'] = df[['score_cleanliness', 'score_comfort', 'score_facilities',
                        'score_location', 'score_staff', 'score_value_for_money']].mean(axis=1)
df['score_min'] = df[['score_cleanliness', 'score_comfort', 'score_facilities',
                       'score_location', 'score_staff', 'score_value_for_money']].min(axis=1)
df['score_max'] = df[['score_cleanliness', 'score_comfort', 'score_facilities',
                       'score_location', 'score_staff', 'score_value_for_money']].max(axis=1)

# 2. Quality indicators
df['high_overall'] = (df['score_overall'] >= 9.0).astype(int)
df['high_value'] = (df['score_value_for_money'] >= 8.5).astype(int)
df['high_location'] = (df['score_location'] >= 9.0).astype(int)

# 3. Interaction features
df['quality_value_ratio'] = df['score_overall'] / (df['score_value_for_money'] + 0.1)
df['location_importance'] = df['score_location'] / (df['score_mean'] + 0.1)
df['staff_quality'] = df['score_staff'] * df['star_rating']

# 4. Hotel-level aggregated features (THIS IS KEY!)
# Aggregate statistics per hotel
hotel_agg = df.groupby('hotel_id').agg({
    'score_overall': ['mean', 'std', 'count'],
    'score_cleanliness': 'mean',
    'score_facilities': 'mean',
    'score_location': 'mean',
    'star_rating': 'first',  # Star rating is same for all reviews of same hotel
}).reset_index()

hotel_agg.columns = ['hotel_id', 'hotel_avg_score', 'hotel_score_std', 'hotel_review_count',
                      'hotel_avg_cleanliness', 'hotel_avg_facilities', 'hotel_avg_location',
                      'hotel_star_rating']

# Fill NaN std with 0 (hotels with only 1 review)
hotel_agg['hotel_score_std'] = hotel_agg['hotel_score_std'].fillna(0)

df = df.merge(hotel_agg, on='hotel_id', how='left')

# 5. User demographics aggregated by hotel
user_demo = df.groupby('hotel_id').agg({
    'age_group': lambda x: x.value_counts().index[0],  # Most common age group
}).reset_index()
user_demo.columns = ['hotel_id', 'dominant_age_group']

df = df.merge(user_demo, on='hotel_id', how='left')

# ===== ENCODE CATEGORICAL VARIABLES =====

# Traveler type
df = pd.get_dummies(df, columns=['traveller_type'], prefix='traveler', drop_first=True)

# User country (THIS IS ACTUALLY USEFUL - shows which nationalities review hotels in different regions!)
df = pd.get_dummies(df, columns=['user_country'], prefix='user_country', drop_first=True)

# User gender
df = pd.get_dummies(df, columns=['user_gender'], prefix='gender', drop_first=True)

# Age group - ordinal encoding
age_order = {'18-24': 1, '25-34': 2, '35-44': 3, '45-54': 4, '55+': 5}
df['age_numeric'] = df['age_group'].map(age_order)

# Dominant age group at hotel
df['dominant_age_numeric'] = df['dominant_age_group'].map(age_order)

# ===== FEATURE SELECTION =====

# Individual review scores
review_score_features = [
    'score_overall', 'score_cleanliness', 'score_comfort', 'score_facilities',
    'score_location', 'score_staff', 'score_value_for_money'
]

# Engineered score features
engineered_score_features = [
    'score_std', 'score_mean', 'score_min', 'score_max',
    'high_overall', 'high_value', 'high_location',
    'quality_value_ratio', 'location_importance', 'staff_quality'
]

# Hotel-level features (CRITICAL!)
hotel_features = [
    'star_rating', 'hotel_avg_score', 'hotel_score_std', 'hotel_review_count',
    'hotel_avg_cleanliness', 'hotel_avg_facilities', 'hotel_avg_location',
    'hotel_star_rating', 'dominant_age_numeric'
]

# User demographics
user_features = ['age_numeric']

# Get all one-hot encoded columns
traveler_cols = [col for col in df.columns if col.startswith('traveler_')]
user_country_cols = [col for col in df.columns if col.startswith('user_country_')]
gender_cols = [col for col in df.columns if col.startswith('gender_')]

# Combine all features
feature_columns = (review_score_features + engineered_score_features + 
                   hotel_features + user_features + 
                   traveler_cols + user_country_cols + gender_cols)

# Verify all features exist
feature_columns = [col for col in feature_columns if col in df.columns]

print(f"\nTotal features: {len(feature_columns)}")
print(f"\nFeature breakdown:")
print(f"  - Review scores: {len(review_score_features)}")
print(f"  - Engineered scores: {len(engineered_score_features)}")
print(f"  - Hotel features: {len([col for col in hotel_features if col in df.columns])}")
print(f"  - Traveler types: {len(traveler_cols)}")
print(f"  - User countries: {len(user_country_cols)}")
print(f"  - Gender: {len(gender_cols)}")

X = df[feature_columns]
y = df['country_group']

# ===== TRAIN-TEST SPLIT =====
# Stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# ===== FEATURE SCALING =====
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===== MODEL 1: IMPROVED LOGISTIC REGRESSION =====
print("\n" + "="*60)
print("IMPROVED LOGISTIC REGRESSION")
print("="*60)

log_model = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',  # Handle class imbalance
    C=0.5,  # Regularization
    solver='saga',
    random_state=42,
    verbose=0
)

log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)

print(f"Accuracy: {accuracy_score(y_test, y_pred_log):.4f}")
print(f"Precision (weighted): {precision_score(y_test, y_pred_log, average='weighted', zero_division=0):.4f}")
print(f"Recall (weighted): {recall_score(y_test, y_pred_log, average='weighted'):.4f}")
print(f"F1 Score (weighted): {f1_score(y_test, y_pred_log, average='weighted'):.4f}")
print("\nPer-class Classification Report:")
print(classification_report(y_test, y_pred_log, zero_division=0))

# ===== MODEL 2: RANDOM FOREST (Better for this task) =====
print("\n" + "="*60)
print("RANDOM FOREST CLASSIFIER")
print("="*60)

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_train, y_train)  # No scaling needed for RF
y_pred_rf = rf_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision (weighted): {precision_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")
print(f"Recall (weighted): {recall_score(y_test, y_pred_rf, average='weighted'):.4f}")
print(f"F1 Score (weighted): {f1_score(y_test, y_pred_rf, average='weighted'):.4f}")
print("\nPer-class Classification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))

# ===== FEATURE IMPORTANCE =====
print("\n" + "="*60)
print("TOP 25 MOST IMPORTANT FEATURES (Random Forest)")
print("="*60)

feature_importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance_df.head(25).to_string(index=False))

# ===== ANALYSIS OF RESULTS =====
print("\n" + "="*60)
print("KEY INSIGHTS")
print("="*60)

# Check class balance
print("\nClass distribution in test set:")
print(y_test.value_counts().sort_index())

# Compare to baseline (most frequent class)
most_frequent_class = y_train.value_counts().index[0]
baseline_accuracy = (y_test == most_frequent_class).sum() / len(y_test)
print(f"\nBaseline accuracy (predicting most frequent class): {baseline_accuracy:.4f}")
print(f"Logistic Regression improvement: {(accuracy_score(y_test, y_pred_log) - baseline_accuracy):.4f}")
print(f"Random Forest improvement: {(accuracy_score(y_test, y_pred_rf) - baseline_accuracy):.4f}")

Total reviews: 50000

Country group distribution:
country_group
Africa                   6132
East_Asia                6082
Eastern_Europe           1970
Middle_East              3983
North_America            3962
North_America_Mexico     2004
Oceania                  4014
South_America            3918
South_Asia               1989
Southeast_Asia           4070
Western_Europe          11876
Name: count, dtype: int64

Total features: 56

Feature breakdown:
  - Review scores: 7
  - Engineered scores: 10
  - Hotel features: 9
  - Traveler types: 3
  - User countries: 24
  - Gender: 2

Training samples: 40000
Test samples: 10000

IMPROVED LOGISTIC REGRESSION
Accuracy: 0.9720
Precision (weighted): 0.9763
Recall (weighted): 0.9720
F1 Score (weighted): 0.9729

Per-class Classification Report:
                      precision    recall  f1-score   support

              Africa       1.00      1.00      1.00      1226
           East_Asia       0.99      1.00      1.00      1216
      Eastern_Eu

In [6]:
# df.rename(columns={"country_x":"hotel_country","country_y":"user_country"},inplace=True)

In [7]:
# groups={'North_America':['United States','Canada'],
#         'Western_Europe':['Germany','France','United Kingdom','Netherlands','Spain','Italy'],
#         'Eastern_Europe':['Russia'],
#         'East_Asia':['China','Japan','South Korea'],
#         'Southeast_Asia':['Thailand','Singapore'],
#         'Middle_East':['United Arab Emirates','Turkey'],
#         'Africa':['Egypt','Nigeria','South Africa'],
#         'Oceania':['Australia','New Zealand'],
#         'South_America':['Brazil','Argentina'],
#         'South_Asia':['India'],
#         'North_America_Mexico':['Mexico']}

# df["country_group"]=df["hotel_country"].apply(lambda x: next((key for key, value in groups.items() if x in value), "Other"))

# df[["hotel_country","user_country","country_group"]]

In [8]:
city_scores = df.groupby(['traveller_type', 'city'])['score_overall'].mean().reset_index().sort_values(['traveller_type', 'score_overall'], ascending=[True,False])

best_cities = city_scores.groupby('traveller_type').head(1)

# display(city_scores)


KeyError: 'traveller_type'

In [ ]:
df = pd.get_dummies(df, columns=['traveller_type'], drop_first=True)
df = pd.get_dummies(df, columns=['user_country'], drop_first=True)
df = pd.get_dummies(df, columns=['user_gender'], drop_first=True)

age_order = {
    '18-24': 1,
    '25-34': 2,
    '35-44': 3,
    '45-54': 4,
    '55+': 5
}

df['age'] = df['age_group'].map(age_order)
df.drop(columns=['age_group'], inplace=True)

: 

In [ ]:

from typing import final


columns_to_drop = [
    'review_id',         
    # 'user_id',           
    # 'hotel_id',          
    'review_date',      
    'join_date',          
    'review_text',       
    'hotel_name',        
    'hotel_country',      
    'lat',                
    'lon'  
]
final_df=df.drop(columns=columns_to_drop)

# final_df.to_csv('final_dataset.csv', index=False)   
final_df.head()


,user_id,hotel_id,score_overall,score_cleanliness,score_comfort,score_facilities,score_location,score_staff,score_value_for_money,city,...,user_country_South Korea,user_country_Spain,user_country_Thailand,user_country_Turkey,user_country_United Arab Emirates,user_country_United Kingdom,user_country_United States,user_gender_Male,user_gender_Other,age
0,1600,1,8.7,8.6,8.7,8.5,9.0,8.8,8.7,New York,...,False,False,False,False,False,False,False,False,False,2
1,432,4,9.1,10.0,9.1,9.0,8.6,9.4,8.6,Tokyo,...,False,False,False,False,False,True,False,False,False,3
2,186,18,8.8,9.7,8.8,8.3,8.7,8.1,8.6,Cairo,...,False,True,False,False,False,False,False,False,False,5
3,1403,19,8.9,9.0,8.8,8.5,9.6,9.1,8.3,Barcelona,...,False,False,False,False,False,True,False,False,False,3
4,1723,17,9.1,8.9,9.5,9.3,8.3,9.4,8.9,Moscow,...,False,False,False,False,False,False,False,True,False,4


: 

In [ ]:

final_df.to_csv('final_dataset.csv', index=False)

: 

In [ ]:
X = final_df[['score_overall','score_cleanliness','score_comfort','score_facilities','score_location','score_staff','score_value_for_money','star_rating','traveller_type_Couple','traveller_type_Family','traveller_type_Solo',"user_country_Australia","user_country_Brazil","user_country_Canada","user_country_China","user_country_Egypt","user_country_France","user_country_Germany","user_country_India","user_country_Italy","user_country_Japan","user_country_Mexico","user_country_Netherlands","user_country_New Zealand","user_country_Nigeria","user_country_Russia","user_country_Singapore","user_country_South Africa","user_country_South Korea","user_country_Spain","user_country_Thailand","user_country_Turkey","user_country_United Arab Emirates","user_country_United Kingdom","user_country_United States", 'user_gender_Male','user_gender_Other','age' ]] 
y = final_df['country_group']

: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

: 

In [ ]:
log_model = LogisticRegression(
    max_iter=1000,
)
log_model.fit(X_train,y_train)

c:\Users\abdel\fire_env_tf\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


: 

In [ ]:
y_pred = log_model.predict(X_test)

print("=== Logistic Regression Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("\nDetailed Report:\n", classification_report(y_test, y_pred))

=== Logistic Regression Evaluation ===
Accuracy: 0.3417
Precision: 0.2854794296385947
Recall: 0.3417
F1 Score: 0.2745063686699206

Detailed Report:
                       precision    recall  f1-score   support

              Africa       0.31      0.47      0.37      1247
           East_Asia       0.34      0.47      0.39      1195
      Eastern_Europe       0.20      0.03      0.05       358
         Middle_East       0.30      0.05      0.09       816
       North_America       0.21      0.10      0.13       776
North_America_Mexico       0.33      0.05      0.09       400
             Oceania       0.00      0.00      0.00       805
       South_America       0.29      0.21      0.24       813
          South_Asia       0.00      0.00      0.00       389
      Southeast_Asia       0.41      0.17      0.24       818
      Western_Europe       0.37      0.76      0.50      2383

            accuracy                           0.34     10000
           macro avg       0.25      0.21  

c:\Users\abdel\fire_env_tf\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abdel\fire_env_tf\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abdel\fire_env_tf\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abdel\fire_en

: 